In [215]:
###Importation des différents librairies

import numpy as np
import plotly.graph_objects as go
import pandas as pd

In [216]:
### initialisation des variables



# --- Paramètres généraux ---
Te = 1           # période d'échantillonnage (capteur)
T = 100          # nombre d'instants (longueur du scénario)

# --- Paramètres de bruit ---
sigma_Q = 1      # écart-type du bruit de processus (modèle dynamique)
sigma_px = 30    # écart-type du bruit de mesure en abscisse (capteur)
sigma_py = 30    # écart-type du bruit de mesure en ordonnée (capteur)

# --- Matrice F (transition d’état) ---
F = np.array([
    [1, Te, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, Te],
    [0, 0, 0, 1]
])

# --- Matrice Q (covariance du bruit de processus) ---
Q = sigma_Q**2 * np.array([
    [Te**3 / 3, Te**2 / 2, 0, 0],
    [Te**2 / 2, Te, 0, 0],
    [0, 0, Te**3 / 3, Te**2 / 2],
    [0, 0, Te**2 / 2, Te]
])

# --- Matrice H (observation) ---
H = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0]
])

# --- Matrice R (covariance du bruit de mesure) ---
R = np.array([
    [sigma_px**2, 0],
    [0, sigma_py**2]
])

# --- État initial ---
x_init = np.array([3, 40, -4, 20])     # état initial de la cible
x_kalm = x_init.copy()                # estimation initiale (x̂₀|₀)
P_kalm = np.eye(4)                    # matrice de covariance initiale (P₀|₀)


In [217]:
#---Création de la fonction qui simule l'équation d'état
def creer_trajectoire(F, Q, x_init, T):
    """
    Génère une trajectoire aléatoire de dimension (4 x T)
    à partir du modèle dynamique X_k = F X_{k-1} + U_k.

    Paramètres :
    - F : matrice de transition d'état (4x4)
    - Q : matrice de covariance du bruit de processus (4x4)
    - x_init : vecteur d'état initial (4,)
    - T : nombre d'instants

    Retourne :
    - x : matrice des états simulés de dimension (4 x T)
    """
    x = np.zeros((4, T))
    x[:, 0] = x_init  # initialisation au temps k=0

    for k in range(1, T):
        bruit = np.random.multivariate_normal(mean=np.zeros(4), cov=Q) # bruit de processus gaussien
        #bruit = np.random.uniform(low=-10, high=10, size=4) # bruit uniforme
        x[:, k] = F @ x[:, k-1] + bruit

    return x

In [218]:
#Appel de la fonction pour créer la trajectoire
x = creer_trajectoire(F, Q, x_init, T) 


In [219]:
# --- Simulation de l'équation d'observation ---

def creer_observation(H, R, x):
    """
    Génère une matrice d'observation à partir du modèle d'observation Y_k = H X_k + V_k.

    Paramètres :
    - H : matrice d'observation (2x4)
    - R : matrice de covariance du bruit de mesure (2x2)
    - x : matrice des états simulés (4xT)

    Retourne :
    - y : matrice des observations simulées (2xT)
    """
    T = x.shape[1]
    y = np.zeros((2, T))

    for k in range(T):
        bruit = np.random.multivariate_normal(mean=np.zeros(2), cov=R) # bruit de mesure gaussien
        #bruit = np.random.uniform(low=0, high=20, size=2) # bruit uniforme
        y[:, k] = H @ x[:, k] + bruit

    return y

In [220]:
# Appel de la fonction pour créer les observations
y = creer_observation(H, R, x)

In [221]:
#Visualisation de la trajectoire et des observations
fig = go.Figure()

# Trajectoire vraie
fig.add_trace(go.Scatter(
    x=x[0], y=x[2],
    mode='lines+markers',
    name='Trajectoire vraie',
    line=dict(color='blue')
))

# Trajectoire observée
fig.add_trace(go.Scatter(
    x=y[0], y=y[1],
    mode='markers',
    name='Observations bruitées',
    marker=dict(color='red', size=6, symbol='circle')
))

fig.update_layout(
    title='Trajectoire vraie vs Observée (Filtrage de Kalman - données synthétiques)',
    xaxis_title='Position x',
    yaxis_title='Position y',
    legend=dict(x=0, y=1),
    width=800,
    height=600
)

fig.show()

In [222]:
def filtre_de_kalman(F, Q, H, R, y_k, x_kalm_prec, P_kalm_prec):
    """
    Effectue une itération du filtre de Kalman (prédiction + mise à jour).

    Entrées :
    - F : matrice de transition (4x4)
    - Q : matrice de covariance du bruit de processus (4x4)
    - H : matrice d’observation (2x4)
    - R : matrice de covariance du bruit de mesure (2x2)
    - y_k : observation actuelle (2,)
    - x_kalm_prec : estimation précédente de l’état (4,)
    - P_kalm_prec : estimation précédente de la covariance (4x4)

    Sorties :
    - x_kalm_k : estimation actuelle de l’état (4,)
    - P_kalm_k : covariance actuelle de l’état (4x4)
    """

    # === Étape 1 : Prédiction ===
    x_pred = F @ x_kalm_prec
    P_pred = F @ P_kalm_prec @ F.T + Q

    # === Étape 2 : Gain de Kalman ===
    S = H @ P_pred @ H.T + R                # Matrice d’innovation
    K = P_pred @ H.T @ np.linalg.inv(S)     # Gain de Kalman

    # === Étape 3 : Mise à jour ===
    y_pred = H @ x_pred                     # observation prédite
    innovation = y_k - y_pred               # différence entre observation réelle et prédite

    x_kalm_k = x_pred + K @ innovation      # estimation corrigée
    P_kalm_k = (np.eye(len(F)) - K @ H) @ P_pred  # mise à jour de la covariance

    return x_kalm_k, P_kalm_k

In [223]:
# --- Initialisation pour le filtrage récursif ---
x_est = np.zeros((4, T))           # Pour mémoriser les états estimés x̂_{k|k}
x_kalm_prec = x_init.copy()        # x̂_{0|0} : estimation initiale
P_kalm_prec = np.eye(4)            # P_{0|0} : covariance initiale

# --- Boucle principale de filtrage ---
for k in range(T):
    # Étape de filtrage de Kalman
    x_kalm_k, P_kalm_k = filtre_de_kalman(F, Q, H, R, y[:, k], x_kalm_prec, P_kalm_prec)
    
    # Mémorisation de l’état estimé à l’instant k
    x_est[:, k] = x_kalm_k
    
    # Mise à jour pour l’itération suivante
    x_kalm_prec = x_kalm_k
    P_kalm_prec = P_kalm_k


In [224]:
# --- Calcul de l’erreur quadratique à chaque instant ---
err_quadra = np.zeros(T)

for k in range(T):
    diff = x[:, k] - x_est[:, k]
    err_quadra[k] = diff.T @ diff   # ||x_k - x̂_k||^2

# --- Moyenne de l’erreur quadratique (avec racine) ---
erreur_moyenne = np.mean(np.sqrt(err_quadra))

# --- Affichage du résultat ---
print(f"Erreur quadratique moyenne : {erreur_moyenne:.4f}")


Erreur quadratique moyenne : 20.6442


In [225]:
# --- Courbe d'erreur quadratique moyenne instantanée ---
fig = go.Figure()

# Courbe des √erreurs à chaque instant
fig.add_trace(go.Scatter(
    y=np.sqrt(err_quadra),
    x=np.arange(T),
    mode='lines',
    name='√Erreur quadratique instantanée',
    line=dict(color='orange')
))

# Ligne horizontale = erreur moyenne
fig.add_trace(go.Scatter(
    y=[erreur_moyenne] * T,
    x=np.arange(T),
    mode='lines',
    name=f'Erreur quadratique moyenne : {erreur_moyenne:.2f}',
    line=dict(dash='dash', color='red')
))

fig.update_layout(
    title="Erreur quadratique (racine) au fil du temps",
    xaxis_title="Temps k",
    yaxis_title="√Erreur quadratique",
    legend=dict(x=0, y=1),
    width=800,
    height=500,
    grid=dict(rows=1, columns=1)
)

fig.show()

In [226]:


temps = np.arange(T)

# --- 1. Position en abscisse (x) ---
fig_x = go.Figure()
fig_x.add_trace(go.Scatter(x=temps, y=x[0], mode='lines', name='Position vraie x', line=dict(color='blue')))
fig_x.add_trace(go.Scatter(x=temps, y=y[0], mode='markers', name='Observation x', marker=dict(color='red', size=4)))
fig_x.add_trace(go.Scatter(x=temps, y=x_est[0], mode='lines', name='Estimation Kalman x', line=dict(color='green')))

fig_x.update_layout(
    title='Position en abscisse (x) dans le temps',
    xaxis_title='Temps k',
    yaxis_title='Position x',
    width=800,
    height=400
)
fig_x.show()

# --- 2. Position en ordonnée (y) ---
fig_y = go.Figure()
fig_y.add_trace(go.Scatter(x=temps, y=x[2], mode='lines', name='Position vraie y', line=dict(color='blue')))
fig_y.add_trace(go.Scatter(x=temps, y=y[1], mode='markers', name='Observation y', marker=dict(color='red', size=4)))
fig_y.add_trace(go.Scatter(x=temps, y=x_est[2], mode='lines', name='Estimation Kalman y', line=dict(color='green')))

fig_y.update_layout(
    title='Position en ordonnée (y) dans le temps',
    xaxis_title='Temps k',
    yaxis_title='Position y',
    width=800,
    height=400
)
fig_y.show()

# --- 3. Trajectoires dans le plan (x, y) ---
fig_xy = go.Figure()
fig_xy.add_trace(go.Scatter(x=x[0], y=x[2], mode='lines+markers', name='Trajectoire vraie', line=dict(color='blue')))
fig_xy.add_trace(go.Scatter(x=y[0], y=y[1], mode='markers', name='Observations', marker=dict(color='red', size=4)))
fig_xy.add_trace(go.Scatter(x=x_est[0], y=x_est[2], mode='lines', name='Trajectoire estimée (Kalman)', line=dict(color='green')))

fig_xy.update_layout(
    title='Trajectoires dans le plan (x, y)',
    xaxis_title='Position x',
    yaxis_title='Position y',
    width=800,
    height=600
)
fig_xy.show()


In [227]:
# Étude combinée de l'influence de sigma_Q et sigma_px = sigma_py sur l'erreur quadratique moyenne

sigma_Q_list = [0.01, 0.1, 1, 5, 10]
sigma_p_list = [1, 10, 30, 100, 200]

resultats = []

for sigma_Q in sigma_Q_list:
    for sigma_p in sigma_p_list:
        Q = sigma_Q**2 * np.array([
            [Te**3/3, Te**2/2, 0, 0],
            [Te**2/2, Te, 0, 0],
            [0, 0, Te**3/3, Te**2/2],
            [0, 0, Te**2/2, Te]
        ])
        R = np.array([[sigma_p**2, 0], [0, sigma_p**2]])
        
        x = creer_trajectoire(F, Q, x_init, T)
        y = creer_observation(H, R, x)
        
        x_est = np.zeros((4, T))
        x_kalm_prec = x_init.copy()
        P_kalm_prec = np.eye(4)
        
        for k in range(T):
            x_kalm_k, P_kalm_k = filtre_de_kalman(F, Q, H, R, y[:, k], x_kalm_prec, P_kalm_prec)
            x_est[:, k] = x_kalm_k
            x_kalm_prec = x_kalm_k
            P_kalm_prec = P_kalm_k

        err_quadra = np.array([np.linalg.norm(x[:, k] - x_est[:, k])**2 for k in range(T)])
        erreur_moyenne = np.mean(np.sqrt(err_quadra))
        resultats.append((sigma_Q, sigma_p, erreur_moyenne))

# Affichage des résultats

df = pd.DataFrame(resultats, columns=["sigma_Q", "sigma_px = sigma_py", "erreur_moyenne"])
df_pivot = df.pivot(index="sigma_Q", columns="sigma_px = sigma_py", values="erreur_moyenne")
df


,sigma_Q,sigma_px = sigma_py,erreur_moyenne
0,0.01,1,2.217095
1,0.01,10,19.109848
2,0.01,30,24.055373
3,0.01,100,21.134452
4,0.01,200,37.726216
5,0.10,1,1.702929
6,0.10,10,7.788044
7,0.10,30,12.433248
8,0.10,100,32.126211
9,0.10,200,53.538898


In [228]:
# Créer la heatmap à partir du tableau pivot
heatmap = go.Figure(data=go.Heatmap(
    z=df_pivot.values,
    x=df_pivot.columns.tolist(),
    y=df_pivot.index.tolist(),
    colorscale='Viridis',   
    colorbar=dict(title="Erreur moyenne")
))

heatmap.update_layout(
    title="Erreur quadratique moyenne selon σQ et σpx = σpy",
    xaxis_title="Bruit de mesure (σpx = σpy)",
    yaxis_title="Bruit de processus (σQ)",
    width=800,
    height=500
)

heatmap.show()
